# Text preprocessing - replace umlauts

In [1]:
def umlauts(word):
    """
    Replace umlauts for a given text
    
    :param word: text as string
    :return: manipulated text as str
    """
    
    tempVar = word  
    
    tempVar = tempVar.replace('ä', 'ae')
    tempVar = tempVar.replace('ö', 'oe')
    tempVar = tempVar.replace('ü', 'ue')
    tempVar = tempVar.replace('Ä', 'Ae')
    tempVar = tempVar.replace('Ö', 'Oe')
    tempVar = tempVar.replace('Ü', 'Ue')
    tempVar = tempVar.replace('ß', 'ss')
    
    return tempVar

In [2]:
with open('test.txt', encoding='utf-8') as file:
    text = file.read()
text = umlauts(text)
text = ' '.join((text.strip('\n').split()))

text_file = open("test.txt", "w")
n = text_file.write(text)
text_file.close()

# hier beginnt die eigentliche BD-Anwendung

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
# if we would use kubernetes then run this instead of next cell
'''conf=SparkConf()\
        .setMaster(local[*])\
        .setappName("WordCount")\
        .setExecutorEnv("spark.executor.memory","4g")\
        .setExecutorEnv("spark.driver.memory","4g")
  spark=SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()'''

In [4]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()
sc=spark.sparkContext

In [5]:
#hier eine txt der Datenbank einhängen für den wordcount
file = 'test.txt'
df = sc.textFile(file)

In [6]:
# exclude punctuation
def lower_clean_str(x):
    punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str

In [7]:
#remove punktuation and transform to lowercase
df = df.map(lower_clean_str) 

In [8]:
#split sentences into list of words
df = df.flatMap(lambda satir: satir.split(" ")) 

In [9]:
# exclude whitespaces
df = df.filter(lambda x:x!='') 

In [11]:
# count how many times each word occurs
count = df.map(lambda word:(word,1))
countRBK = count.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [13]:
# rank words
countRBK = countRBK.map(lambda x:(x[1],x[0]))
countRBK = countRBK.sortByKey(False)

In [15]:
# exclude stopwords
## if normal importing doesn't work:   ##
## python -m nltk.downloader stopwords ##

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords =stopwords.words('german')

german_stopwords = []
for word in stopwords:
    german_stopwords.append(umlauts(word))

countRBK = countRBK.filter(lambda x: x[1] not in german_stopwords)

In [17]:
countRBK.take(10)

[(8, 'internet'),
 (8, 'sagte'),
 (6, 'bett'),
 (6, 'frau'),
 (6, 'gesicht'),
 (5, 'beim'),
 (5, 'fenster'),
 (5, 'ganz'),
 (5, 'hand'),
 (5, 'hoch')]

In [78]:
import pandas as pd
test = pd.read_csv('./../output/app_test.txt')

test

,{'site': 'faz','time': '13:45:51','title': 'Drohende Insolvenzen: Wie deutsche Gastwirte vor dem Winter zittern','link': 'https://www.faz.net/-gqi-a2z0k','text': 'Die Marmortische in „Elaine’s Restaurant“ schmiegen sich noch aneinander als gäbe es kein Corona als wäre die Eröffnung eines Dinner-Lokals für After-Work- und Pre-Party-Gäste gerade kein „heißer Ritt“ wie Luel Mulugeta gerne sagt. Meistens lacht er dabei den rechten Fuß mit den bunten Nike-Sneakern lässig übereinandergeschlagen. Er und sein Geschäftspartner Guy Lamaye expandieren mitten in der Krise und in großem Stil. Sie haben einen Live-Zähler auf ihrer neuen Website eingebaut: Er zeigt an wie viele Mitarbeiter sie schon haben – aktuell sind es 94 inklusive Aushilfen. Jetzt darf bloß kein zweiter Lockdown kommen. „Dann würden wir kaputtgehen“ sagt Lamaye und lächelt ein bisschen nervös. „Aber jetzt machen wir es erst mal Covid-Style“ unterbricht ihn Mulugeta.In drei Tagen werden in „Elaine’s Restaurant“ die ersten Gäste sitzen. Es ist nicht die erste Location und es wird nicht die letzte sein die Mulugeta und Lamaye – ursprünglich Betreiber eines einzigen Cafés im Zentrum der Stadt – während der Pandemie in Frankfurt eröffnen. Eine zweite Cafébar hat im Mai aufgemacht sie läuft noch nicht so recht. Drei weitere Lokale kommen noch die Mietverträge sind unterschrieben die Einrichtung gekauft die Mitarbeiter eingestellt.','authors': ['Sarah Obertreis'],'tags': ['wirtschaft']}


In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   {'col_1': "3"  0 non-null      object
 1    'col_2': 'a   0 non-null      object
 2   b b c          0 non-null      object
 3   nj njnj. iono  0 non-null      object
 4   nnjk.d'}       0 non-null      object
dtypes: object(5)
memory usage: 80.0+ bytes
